In [1]:
!pip install -U -q PyDrive


In [5]:
'''from google.colab import drive
drive.mount('/content/drive')


from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)'''


Mounted at /content/drive


In [6]:
'''link = 'https://drive.google.com/file/d/1l_5o4Bkbs3Uab_WezeCoo2C3JCb8BrBc/view'

import pandas as pd

# to get the id part of the file
id = link.split("/")[-2]
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('train.tsv')
train = pd.read_csv('train.tsv', sep = '\t')
#########
link = 'https://drive.google.com/file/d/1ID_g_xHfJ2NkXLIxAnL9m05pCaDHSteH/view'
id = link.split("/")[-2]
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('test.tsv')

test = pd.read_csv('test.tsv', sep = '\t')'''

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet') #Testing 123

#https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rishub\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Rishub\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rishub\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rishub\AppData\Roaming\nltk_data...


True

In [3]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

# Step 2: Download NLTK resources
nltk.download('punkt')  # For tokenization
nltk.download('averaged_perceptron_tagger')  # For POS tagging (optional)
nltk.download('stopwords')  # For stopword removal
nltk.download('wordnet')  # For lemmatization
nltk.download('punkt_tab')
nltk.download('all')

train = pd.read_csv('train.tsv', sep = '\t')
test = pd.read_csv('test.tsv', sep = '\t')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rishub\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Rishub\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rishub\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rishub\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Rishub\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Rishub\AppData\Roaming\nlt

In [4]:

np.random.seed(123)
X_train = train.iloc[:, 0:-1]
y_train = train.iloc[:, -1]
x_test = test


X_train['Phrase'].dropna(inplace=True) #remove blank rows
X_train['Phrase'] = [word.lower() for word in X_train['Phrase']] #make everything lowercase
X_train['Phrase'] = [word_tokenize(pharse) for pharse in X_train['Phrase']] #Each phrase is broken into a set of words


#
#All FOR TRAINING DATA
#

#Trying to see if each word is a noun, verb or adjective
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

for index,entry in enumerate(X_train['Phrase']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    X_train.loc[index,'text_final'] = str(Final_words)

#
#SAME THING FOR TEST DATA]
X_test = test
X_test['Phrase'].dropna(inplace=True) #remove blank rows
X_test['Phrase'] = X_test['Phrase'].astype(str)
X_test['Phrase'] = [word.lower() for word in X_test['Phrase']] #make everything lowercase
X_test['Phrase'] = [word_tokenize(pharse) for pharse in X_test['Phrase']] #Each phrase is broken into a set of words

for index,entry in enumerate(X_test['Phrase']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    X_test.loc[index,'text_final'] = str(Final_words)






In [5]:
#y_train = y_train.to_numpy() #Turn the sentiment scores of the df into an array

print(len(X_train))
print(len(y_train))
# Convert lists of tokens into strings
X_train['text_final'] = X_train['text_final'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
X_test['text_final'] = X_test['text_final'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

y_train.dropna(inplace=True)

Encoder = LabelEncoder()
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(y_train)

print(X_train['text_final'].apply(type).unique())

# Remove rows where 'text_final' is an empty string or contains only whitespace
X_train = X_train[X_train['text_final'].apply(lambda x: isinstance(x, str) and len(x.strip()) > 0)]
y_train = y_train[X_train.index]  # Ensure y_train is aligned with X_train after filtering

# Now, confirm the lengths again
print("Shape of X_train:", X_train.shape)
print("Length of y_train:", len(y_train))





156060
156060
[<class 'str'>]
Shape of X_train: (156060, 4)
Length of y_train: 156060


In [6]:
# Re-run the TF-IDF vectorization after ensuring that 'text_final' is valid
Tfidf_vect = TfidfVectorizer(max_features=5000)
Train_X_Tfidf = Tfidf_vect.fit_transform(X_train['text_final'])
Test_X_Tfidf = Tfidf_vect.transform(X_test['text_final'])

# Check the shapes after transformation
print("Shape of Train_X_Tfidf:", Train_X_Tfidf.shape)
print("Length of y_train:", len(y_train))




Shape of Train_X_Tfidf: (156060, 5000)
Length of y_train: 156060


In [7]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,y_train)


SVC(gamma='auto', kernel='linear')

In [8]:
# Predict sentiments for the test set
y_test_predictions = SVM.predict(Test_X_Tfidf)

# Assuming `test` dataframe has a column `PhraseId`
submission = pd.DataFrame({
    "PhraseId": test["PhraseId"],  # Use the PhraseId from the test set
    "Sentiment": y_test_predictions  # Use the predictions generated by the model
})

# Save to CSV
submission.to_csv("svmSubmission.csv", index=False)
print("Submission file created!")



Submission file created!
